## Generate single instrument soundtrack


In [1]:
from music21 import instrument

# Instrument list
instruments = {"Piano": instrument.Piano(),
               "Harpsichord": instrument.Harpsichord(),
               "Glockenspiel": instrument.Glockenspiel(),
               "Xylophone": instrument.Xylophone(),
               "ElectricOrgan": instrument.ElectricOrgan(),
               "Harmonica": instrument.Harmonica(),
               "AcousticGuitar": instrument.AcousticGuitar(),
               "ElectricGuitar": instrument.ElectricGuitar(),
               "ElectricBass": instrument.ElectricBass(),
               "Violin": instrument.Violin(),
               "Viola": instrument.Viola(),
               "Violoncello": instrument.Violoncello(),
               "Harp": instrument.Harp(),
               "Timpani": instrument.Timpani(),
               "Trombone": instrument.Trombone(),
               "Tuba": instrument.Tuba(),
               "SopranoSaxophone": instrument.SopranoSaxophone(),
               "Flute": instrument.Flute(),
               "Koto": instrument.Koto(),
               "Soprano": instrument.Soprano()}

# Tempo list
tmpo_fctr = {"original": 1.00,
             "x125": 1.25,
             "x150": 1.50,
             "x075": 0.75,
             "x050": 0.50}
# Key
keys = [-2, -1, 0, 1, 2]

# major conversions
majors = dict([("A-", 4), ("A", 3), ("B-", 2), ("B", 1), ("C", 0), ("D-", -1),
              ("D", -2), ("E-", -3), ("E", -4), ("F", -5), ("G-", 6), ("G", 5)])
minors = dict([("A-", 1), ("A", 0), ("B-", -1), ("B", -2), ("C", -3),
              ("D-", -4), ("D", -5), ("E-", 6), ("E", 5), ("F", 4), ("G-", 3), ("G", 2)])


## Synthesize MIDI to audio (mp3)


In [2]:
from midi2audio import FluidSynth


def midi_to_audio(input_midi, output_audio):
    '''
    Using the default sound font in 44100 Hz sample rate
    '''
    fs = FluidSynth()
    fs.midi_to_audio(input_midi, output_audio)


## Generate single instrument soundtrack


In [3]:
from music21 import converter
from tqdm import tqdm
from pathlib import Path


_TARGET_AUDIO_PATH = rf'audio'
_MIDI_PATH = rf'audio/tmp'
_AUDIO_PATH = rf'audio/instruments'
_MID = rf'.midi'
_MP3 = rf'.mp3'
_MIDI_SOLO_CHOIR = rf'HighwayToHell_solo_choir'

score = converter.parse(Path(_TARGET_AUDIO_PATH, f'{_MIDI_SOLO_CHOIR}{_MID}'))

# Create instrument
for inst_name, inst_obj in tqdm(instruments.items()):
    score.parts[0].insert(0, inst_obj)
    # With diferent tempo
    # scale (in this case stretch) the overall tempo by this factor
    for tmpo_name, tempo in tmpo_fctr.items():
        mod_score = score.scaleOffsets(tempo).scaleDurations(tempo)

        # With diferent key
        for halfstep in keys:
            _SOLO_MIDI = ''
            if halfstep == 0:
                _SOLO_MIDI = rf'HighwayToHell_solo_{inst_name}_{tmpo_name}_original{_MID}'
                mod_score.write('midi', Path(_MIDI_PATH, _SOLO_MIDI))
            else:
                new_key_score = mod_score.transpose(halfstep)
                key = new_key_score.analyze('key')
                
                _SOLO_MIDI = rf'HighwayToHell_solo_{inst_name}_{tmpo_name}_{key.tonic}{key.mode}{_MID}'
                new_key_score.write('midi', Path(_MIDI_PATH, _SOLO_MIDI))

            midi_to_audio(Path(_MIDI_PATH, _SOLO_MIDI), Path(
                _AUDIO_PATH, f'{_SOLO_MIDI[:-5]}{_MP3}'))


FileNotFoundError: Cannot find file in audio/HighwayToHell_solo_choir.midi

## Load a trained neural network model


In [ ]:
import tensorflow as tf
from musicnn_keras.tagger import top_tags


musicnn = tf.keras.models.load_model(
    './musicnn_keras/keras_checkpoints/MSD_musicnn.h5')


In [ ]:
import json

_MP3_PATH = rf'audio/instruments'

audios = Path(_MP3_PATH).glob('*.mp3')
audio_list = []
audio_list.append(str(f).split('/')[2] for f in audios)
audio_json = dict()

for af in tqdm(audios):
    # Get Tags
    audio_dict = dict()
    audio_dict['tags'] = top_tags(str(af),
                                  model='MTT_musicnn',
                                  topN=10,
                                  print_tags=False)

    # Save result
    audio_name = str(af).split('/')[-1]
    meta_data = audio_name.split('.')[0].split('_')
    audio_dict['instrumrnt'] = meta_data[2]
    audio_dict['tempo'] = meta_data[3]
    audio_dict['key'] = meta_data[4]
    audio_json[audio_name.split('.')[0]] = audio_dict

with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(audio_json, f, ensure_ascii=False, indent=4, sort_keys=True)
